In [1]:
import pandas as pd
import numpy as np

import glob
from scipy.stats import spearmanr
from sklearn.metrics.pairwise import euclidean_distances#transform dataframe into rdm

import matplotlib.pyplot as plt
import seaborn as sns

# turn model result into RDM

In [4]:
exp_relationship_df_data_paths = glob.glob('../output_data/individual/category/dimension_results/exp_mds_2V_ind/relationship_score_none/*.csv')
for f in exp_relationship_df_data_paths:
    sub_csv = f.split('\\')[1]
    exp_df = pd.read_csv(f,index_col = 0)
    exp_dissim = pd.DataFrame(euclidean_distances(exp_df),
                              index=exp_df.index,columns=exp_df.index)
    exp_dissim.to_csv('../output_data/individual/category/dimension_results/exp_mds_2V_ind/RDMs_relationship_score_none/'+sub_csv)

In [5]:
imp_relationship_df_data_paths = glob.glob('../output_data/individual/category/dimension_results/imp_mds_2V_ind/relationship_score_none/*.csv')
for f in imp_relationship_df_data_paths:
    sub_csv = f.split('\\')[1]
    imp_df = pd.read_csv(f,index_col = 0)
    imp_dissim = pd.DataFrame(euclidean_distances(imp_df),
                              index=imp_df.index,columns=imp_df.index)
    imp_dissim.to_csv('../output_data/individual/category/dimension_results/imp_mds_2V_ind/RDMs_relationship_score_none/'+sub_csv)

In [6]:
dim_scores_var_df_data_paths = glob.glob('../output_data/individual/dimension/pca_results/scores_var/*.csv')
for f in dim_scores_var_df_data_paths:
    sub_csv = f.split('\\')[1]
    dim_df = pd.read_csv(f,index_col = 0)
    dim_dissim = pd.DataFrame(euclidean_distances(dim_df),
                              index=dim_df.index,columns=dim_df.index)
    dim_dissim.to_csv('../output_data/individual/dimension/pca_results/RDMs_scores_var/'+sub_csv)

# run loocv

In [7]:
def LOOCV(subj_data_paths):
    rsa_list = []
    sub_id_list = []
    for left_one in subj_data_paths:
        sub_id_list.append(left_one.split('\\')[1].split('.')[0])
        # get the one left
        left_one_df = pd.read_csv(left_one, index_col=0)
        dissim_left_one = left_one_df
        
        # remove one subjects from all 10 subjects
        subj_data_paths_remove_one = subj_data_paths.copy()
        subj_data_paths_remove_one.remove(left_one)
        
        dissim_remove_one = np.zeros((159,159))

        for f in subj_data_paths_remove_one:
            temp_df = pd.read_csv(f, index_col=0)
            dissim_remove_one += np.array(temp_df)

        dissim_remove_one_avg = dissim_remove_one / len(subj_data_paths_remove_one)
        dissim_remove_one_avg = pd.DataFrame(dissim_remove_one_avg, index=temp_df.index, columns=temp_df.columns)
        
        # caculate the correlation between the one left and others
        array_dissim_left_one = np.array(dissim_left_one)[np.triu_indices(159, k = 1)] 
        array_dissim_remove_one_avg = np.array(dissim_remove_one_avg)[np.triu_indices(159, k = 1)]
           
        rsa_list.append(spearmanr(array_dissim_remove_one_avg, array_dissim_left_one)[0])
    print(len(subj_data_paths_remove_one))
    
    rsa_list_df = pd.DataFrame({'sub_id':sub_id_list,'rsa':rsa_list})
    return rsa_list_df

In [9]:
exp_mds_rdm_data_paths = glob.glob('../output_data/individual/category/dimension_results/exp_mds_2V_ind/RDMs_relationship_score_none/*.csv')
exp_mds_loocv = LOOCV(subj_data_paths = exp_mds_rdm_data_paths)

imp_mds_rdm_data_paths = glob.glob('../output_data/individual/category/dimension_results/imp_mds_2V_ind/RDMs_relationship_score_none/*.csv')
imp_mds_loocv = LOOCV(subj_data_paths = imp_mds_rdm_data_paths)

dim_pca_rdm_data_paths = glob.glob('../output_data/individual/dimension/pca_results/RDMs_scores_var/*.csv')
dim_pca_loocv = LOOCV(subj_data_paths = dim_pca_rdm_data_paths)

13
13
13


In [11]:
dimension_loocv = pd.merge(pd.merge(exp_mds_loocv,imp_mds_loocv,on=['sub_id']),
                         dim_pca_loocv,on=['sub_id'])
dimension_loocv.columns = ['sub_id','exp','imp','dim']
dimension_loocv.index = dimension_loocv['sub_id']
dimension_loocv = dimension_loocv.drop(columns=['sub_id'])
dimension_loocv.to_csv('../output_data/model_comparison/LOOCV/dimension_model_loocv.csv')